In [83]:
import pandas as pd
import mplfinance as mpf
import sqlalchemy

engine = sqlalchemy.create_engine('sqlite:///C:\\Users\\Michal\\Desktop\\Portfolio_Projects\\Trading_bot\\BTCUSDTsource.db')
df1 = pd.read_sql('btcusdt', engine)

#cos sie stalo z data i jest duuupa :(
# wiec sam dodaje jeden brakujacy wiersz
line = pd.DataFrame({"open":16567.4, "high":16580.0, "low":16555.4, "close":16560, "RSI":45.65}, index=[32])
df1 = pd.concat([df1.iloc[:31], line, df1.iloc[31:]]).reset_index(drop=True)

In [84]:
df1.index = pd.date_range('2022-11-24 17:30:00',periods=(len(df1)), freq='5min')

In [28]:
df1.iloc[5]

open     16600.900000
high     16645.600000
low      16598.800000
close    16612.800000
RSI         59.861072
Name: 2022-11-24 17:55:00, dtype: float64

In [85]:
df1

,open,high,low,close,RSI
2022-11-24 17:30:00,16558.0,16563.2,16506.7,16526.0,43.904435
2022-11-24 17:35:00,16526.1,16590.2,16523.0,16580.1,54.310591
2022-11-24 17:40:00,16580.0,16590.4,16561.9,16581.7,54.578956
2022-11-24 17:45:00,16581.6,16614.9,16570.6,16597.3,57.217511
2022-11-24 17:50:00,16597.4,16610.1,16590.7,16600.8,57.809686
...,...,...,...,...,...
2022-11-25 12:50:00,16515.1,16523.1,16507.9,16519.4,58.810138
2022-11-25 12:55:00,16519.3,16527.5,16518.0,16523.5,59.774282
2022-11-25 13:00:00,16523.5,16527.5,16515.0,16523.0,59.591091
2022-11-25 13:05:00,16522.9,16527.5,16514.5,16523.0,59.591091


In [30]:
df1.iloc[[-1]][['high', 'low']]
#df.iloc[[nr_indeksu]] dostaje wiersz

,high,low
2022-11-25 13:10:00,16549.6,16518.1


In [89]:
#slicing dataframe
#df = df.loc[index[0]:]
df1.loc['2022-11-25 12:50:00':]

,open,high,low,close,RSI
2022-11-25 12:50:00,16515.1,16523.1,16507.9,16519.4,58.810138
2022-11-25 12:55:00,16519.3,16527.5,16518.0,16523.5,59.774282
2022-11-25 13:00:00,16523.5,16527.5,16515.0,16523.0,59.591091
2022-11-25 13:05:00,16522.9,16527.5,16514.5,16523.0,59.591091
2022-11-25 13:10:00,16523.0,16549.6,16518.1,16539.5,63.833210


In [59]:
str(df1.iloc[[-1]].index[0])

'2022-11-25 13:10:00'

In [52]:
thisdict = {
  "brand": "Ford",
  "model": "Mustang",
  "year": 1964
}
thisdict['year'] = 2020
thisdict['year']

2020

In [60]:
length = len(df1)
#tworze pomocnizca tablice pozniej jej nie bedzie bo bede operowal na danych z sql
df = pd.DataFrame()

#finding trend - it is all about borders
trend = 'no'
borderHigh = {}
borderLow = {}

#zmienne dla dwoch swiec up i down
numberUp = 0
numberDown = 0
#defining trend with a small amount of data
for i in range(length): #tego fora zamieniam na "while True" i czytanie danych z sql
    df = pd.concat([df, df1.iloc[[i]]]) #to tez pomocnicze (sztuczna rzeczywistosc zbieranie danych )
    if trend == 'no':
        borderHigh = {'time': str(df.iloc[[-1]].index[0]), 'border': float(df.iloc[[-1]]['high'])}
        borderLow = {'time': str(df.iloc[[-1]].index[0]), 'border': float(df.iloc[[-1]]['low'])}
        trend = 'not_found'
        continue
    #finding if new candle dislocate my boarder
    tempHigh = float(df.iloc[[-1]]['high'])
    tempLow = float(df.iloc[[-1]]['low'])
    tempChange = 0
    if tempHigh > borderHigh['border']:
        borderHigh['border'] = tempHigh
        borderHigh['time'] = str(df.iloc[[-1]].index[0])
        trend = 'temp_up'
        tempChange += 1
    if tempLow < borderLow['border']:
        borderLow['border'] = tempLow
        borderLow['time'] = str(df.iloc[[-1]].index[0])
        trend = 'temp_down'
        tempChange += 1
    # unidentified trend
    if tempChange == 2:
        trend = 'not_found'
    #jesli sobie idzie w gore to sprawdzam czy dostane dwie czerwone swieczki, jesli w dol to dwie zielone
    #jak mam dwie swieczki to trend pomocniczy zmienia sie w prawdziwy
    color = candle_color(float(df.iloc[[-1]]['open']), float(df.iloc[[-1]]['close']))
    if trend == 'temp_up':
        if color == 'red':
            numberUp += 1
            if numberUp == 2:
                trend = 'up'
                print(trend)
                print(i)
                print(df.iloc[[-3]])
                
                print(borderHigh)
                print(borderLow)
                break
    if trend == 'temp_down':
        if color == 'green':
            numberDown += 1
            if numberDown == 2:
                trend = 'down'
                print(trend)
                print(i)
                print(df.iloc[[-3]])
                print(borderHigh)
                print(borderLow)
                break
    print(trend)
            
#granice sb przesuwam
#jak wybija granice to jest to pomocniczy trend
#jesli sobie idzie w gore to sprawdzam czy dostane dwie czerwone swieczki, jesli w dol to dwie zielone
#jak mam dwie swieczki to trend pomocniczy zmienia sie w prawdziwy

temp_up
temp_up
temp_up
temp_up
temp_up
temp_up
temp_up
temp_up
up
9
                        open     high      low    close        RSI
2022-11-24 18:05:00  16617.6  16659.9  16616.0  16634.2  63.474135
{'time': '2022-11-24 18:05:00', 'border': 16659.9}
{'time': '2022-11-24 17:30:00', 'border': 16506.7}


In [61]:
def candle_color(o, c):
    if o > c:
        return 'red'
    elif o < c:
        return 'green'
    else:
        return 'undefined'

def firstTrendAndBoarder():
    # finding trend - it is all about borders
    trend = 'no'
    borderHigh = {}
    borderLow = {}
    
    # varuables for candle up and down
    numberUp = 0
    numberDown = 0
    
    #defining first trend and boarder, when the program just started
    
    length = len(df1)
    #tworze pomocnizca tablice pozniej jej nie bedzie bo bede operowal na danych z sql
    df = pd.DataFrame()
    for i in range(length): #tego fora zamieniam na "while True" i czytanie danych z sql
        df = pd.concat([df, df1.iloc[[i]]]) #to tez pomocnicze (sztuczna rzeczywistosc zbieranie danych )
        
        if trend == 'no':
            borderHigh = {'time': str(df.iloc[[-1]].index[0]), 'border': float(df.iloc[[-1]]['high'])}
            borderLow = {'time': str(df.iloc[[-1]].index[0]), 'border': float(df.iloc[[-1]]['low'])}
            trend = 'not_found'
            continue
        #finding if new candle dislocate my boarder
        tempHigh = float(df.iloc[[-1]]['high'])
        tempLow = float(df.iloc[[-1]]['low'])
        tempChange = 0
        if tempHigh > borderHigh['border']:
            borderHigh['border'] = tempHigh
            borderHigh['time'] = str(df.iloc[[-1]].index[0])
            trend = 'temp_up'
            tempChange += 1
        if tempLow < borderLow['border']:
            borderLow['border'] = tempLow
            borderHigh['time'] = str(df.iloc[[-1]].index[0])
            trend = 'temp_down'
            tempChange += 1
        # unidentified trend
        if tempChange == 2:
            trend = 'not_found'
        #if it is goung up / down, check if I get two red / green candle
        #if I have to the same candle colors, end function I found trend 
        color = candle_color(float(df.iloc[[-1]]['open']), float(df.iloc[[-1]]['close']))
        if trend == 'temp_up':
            if color == 'red':
                numberUp += 1
                if numberUp == 2:
                    trend = 'up'
                    return [trend, borderHigh, borderLow, i]#to i tylko dlatego ze dzialam na danych ktore juz mam
        if trend == 'temp_down':
            if color == 'green':
                numberDown += 1
                if numberDown == 2:
                    trend = 'down'
                return [trend, borderHigh, borderLow, i]#to i tylko dlatego ze dzialam na danych ktore juz mam

In [62]:
trend,  borderHigh, borderLow, temp_i = firstTrendAndBoarder()

In [63]:
print(trend, borderHigh, borderLow, temp_i )

up {'time': '2022-11-24 18:05:00', 'border': 16659.9} {'time': '2022-11-24 17:30:00', 'border': 16506.7} 9


In [119]:
trend,  borderHigh, borderLow, temp_i = firstTrendAndBoarder()
#   sprawdzam kolejna strzalke czy wybija granice
#       jesli nie wybija to trend zostaje
#       jesli wybija to zmieniam granice i przypisuje odpowiedni trend
#           sprawdzam czy nie jest tuz przy poprzedniej do 3
#                jesli tak to nie nie zmieniam drugiej granicy
#                  jesli nie to:
#                       ustawiam druga granice na min lub max ktore znajduje sie pomiedzy dwoma szczytami lub dolkami
#                       (np jesli wybil nowy wyierzcholek to szukam min miedzy dwoma wierzcholkami i ustawiam granice down)
#   powtarzam cykl
length = len(df1)
    #tworze pomocnizca tablice pozniej jej nie bedzie bo bede operowal na danych z sql
df = pd.DataFrame()
#tej tez nie bedzie bo musze miec wszystkie dane jakie zdazylem pobrac
for i in range(temp_i): #tego fora zamieniam na "while True" i czytanie danych z sql
        df = pd.concat([df, df1.iloc[[i]]]) #to tez pomocnicze (sztuczna rzeczywistosc zbieranie danych )
for i in range(temp_i + 1, length): #pomocnicze, jak bedzie to w funkcji to ona sie wykonuje raz i wychodzi
    df = pd.concat([df, df1.iloc[[i]]]) #wiec do funkcji musze przekazac parametry swieczki 
    
    #finding if new candle dislocate my boarder
    tempHigh = float(df.iloc[[-1]]['high'])
    tempLow = float(df.iloc[[-1]]['low'])
    tempChange = 0
    #borderTime = ''
    if tempHigh > borderHigh['border']:
        borderHigh['border'] = tempHigh
        trend = 'up'
        tempChange += 1
        
        # sprawdzam czy nie jest tuz przy poprzedniej do 3
        if not (borderHigh['time'] == str(df.iloc[[-2]].index[0]) or borderHigh['time'] == str(df.iloc[[-3]].index[0])):
            #ustawiam druga granice na min lub max ktore znajduje sie pomiedzy dwoma szczytami lub dolkami
            dfBetweenBorder = df.loc[borderHigh['time']:]
            #zmienic to border zeby nie bralo od tej co mam tylko nastpnej swieczki
            betweenBorderMin = float(dfBetweenBorder['low'].min()) #nwn czy to musi byc jako float bo jest numpy float
            borderLow['border'] = betweenBorderMin
            print(f"borderLow['time'] = {borderLow['time']}")
            borderLow['time'] = str(dfBetweenBorder[dfBetweenBorder['low'] == betweenBorderMin].index[0])
            print(f"borderLow['time'] = {borderLow['time']}")
            #print(dfBetweenBorder)
            print(f"borderHigh['time'] = {borderHigh['time']}")
            print(str(df.iloc[[-1]].index[0]))
            
            print(f"zmiana tempHigh {trend}")
            

        borderHigh['time'] = str(df.iloc[[-1]].index[0])
        
    if tempLow < borderLow['border']:
        borderLow['border'] = tempLow
        trend = 'down'
        tempChange += 1
        
        if not (borderLow['time'] == str(df.iloc[[-2]].index[0]) or borderLow['time'] == str(df.iloc[[-3]].index[0])):
            #ustawiam druga granice na min lub max ktore znajduje sie pomiedzy dwoma szczytami lub dolkami
            #borderHigh['border'] = 
            dfBetweenBorder = df.loc[borderLow['time']:]
            #zmienic to border zeby nie bralo od tej co mam tylko nastpnej swieczki
            betweenBorderMax = float(dfBetweenBorder['high'].max()) #nwn czy to musi byc jako float bo jest numpy float
            borderHigh['border'] = betweenBorderMax
            print(f"borderHigh['time'] = {borderHigh['time']}")
            borderHigh['time'] = str(dfBetweenBorder[dfBetweenBorder['high'] == betweenBorderMax].index[0])
            print(f"borderHigh['time'] = {borderHigh['time']}")
            #print(dfBetweenBorder)
            print(f"borderLow['time'] = {borderLow['time']}")
            print(str(df.iloc[[-1]].index[0]))
            print(f"zmiana tempLow{trend}")
            
        borderLow['time'] = str(df.iloc[[-1]].index[0])
        
    # unidentified trend
    if tempChange == 2:
        trend = 'undefined'
    print(i)
# dziaaaaaaala

10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
borderHigh['time'] = 2022-11-24 18:05:00
borderHigh['time'] = 2022-11-24 18:05:00
borderLow['time'] = 2022-11-24 17:30:00
2022-11-25 01:50:00
zmiana tempLowdown
100
101
102
103
104
105
106
107
108
109
110
111
112
113
borderHigh['time'] = 2022-11-24 18:05:00
borderHigh['time'] = 2022-11-25 02:25:00
borderLow['time'] = 2022-11-25 01:50:00
2022-11-25 03:00:00
zmiana tempLowdown
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
borderHigh['time'] = 2022-11-25 02:25:00
borderHigh['time'] = 2022-11-25 03:50:00
borderLow['time'] = 2022-11-25 03:00:00
2022-11-25 05:35:00
zmiana tempLowdown
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161

In [110]:
type(float(df.loc[:'2022-11-25 01:50:00']['high'].max()))

float

In [116]:
str(df[df['high'] == df.loc[:'2022-11-25 01:50:00']['high'].max()].index[0])

'2022-11-24 18:05:00'

In [129]:
from datetime import datetime, timedelta

In [131]:
trend,  borderHigh, borderLow, temp_i = firstTrendAndBoarder()

#   sprawdzam kolejna strzalke czy wybija granice
#       jesli nie wybija to trend zostaje
#       jesli wybija to zmieniam granice i przypisuje odpowiedni trend
#           sprawdzam czy nie jest tuz przy poprzedniej do 3
#                jesli tak to nie nie zmieniam drugiej granicy
#                  jesli nie to:
#                       ustawiam druga granice na min lub max ktore znajduje sie pomiedzy dwoma szczytami lub dolkami
#                       (np jesli wybil nowy wyierzcholek to szukam min miedzy dwoma wierzcholkami i ustawiam granice down)
#   powtarzam cykl
length = len(df1)
    #tworze pomocnizca tablice pozniej jej nie bedzie bo bede operowal na danych z sql
df = pd.DataFrame()
#tej tez nie bedzie bo musze miec wszystkie dane jakie zdazylem pobrac
for i in range(temp_i): #tego fora zamieniam na "while True" i czytanie danych z sql
        df = pd.concat([df, df1.iloc[[i]]]) #to tez pomocnicze (sztuczna rzeczywistosc zbieranie danych )
for i in range(temp_i + 1, length): #pomocnicze, jak bedzie to w funkcji to ona sie wykonuje raz i wychodzi
    df = pd.concat([df, df1.iloc[[i]]]) #wiec do funkcji musze przekazac parametry swieczki 
    
    #finding if new candle dislocate my boarder
    tempHigh = float(df.iloc[[-1]]['high'])
    tempLow = float(df.iloc[[-1]]['low'])
    tempChange = 0
    #borderTime = ''
    if tempHigh > borderHigh['border']:
        borderHigh['border'] = tempHigh
        trend = 'up'
        tempChange += 1
        
        # sprawdzam czy nie jest tuz przy poprzedniej do 3
        if not (borderHigh['time'] == str(df.iloc[[-2]].index[0]) or borderHigh['time'] == str(df.iloc[[-3]].index[0])):
            
        #zmieniam to zeby byl between border pomiedzy granicami ale nie wlacznie tylko nawiat otwrty (nie od 5:55 tylko od 6)
            borderFromTime = datetime.strptime(borderHigh['time'], '%Y-%m-%d %H:%M:%S')
            borderFromTime += timedelta(minutes=5)
            print(borderFromTime)
            dfBetweenBorder = df.loc[borderFromTime:]
            #zmienic to border zeby nie bralo od tej co mam tylko nastpnej swieczki
            betweenBorderMin = float(dfBetweenBorder['low'].min()) #nwn czy to musi byc jako float bo jest numpy float
            borderLow['border'] = betweenBorderMin
            print(f"borderLow['time'] = {borderLow['time']}")
            borderLow['time'] = str(dfBetweenBorder[dfBetweenBorder['low'] == betweenBorderMin].index[0])
            print(f"borderLow['time'] = {borderLow['time']}")
            #print(dfBetweenBorder)
            print(f"borderHigh['time'] = {borderHigh['time']}")
            print(str(df.iloc[[-1]].index[0]))
            
            print(f"zmiana tempHigh {trend}")
            

        borderHigh['time'] = str(df.iloc[[-1]].index[0])
        
    if tempLow < borderLow['border']:
        borderLow['border'] = tempLow
        trend = 'down'
        tempChange += 1
        
        if not (borderLow['time'] == str(df.iloc[[-2]].index[0]) or borderLow['time'] == str(df.iloc[[-3]].index[0])):
            borderFromTime = datetime.strptime(borderLow['time'], '%Y-%m-%d %H:%M:%S')
            borderFromTime += timedelta(minutes=5)
            print(borderFromTime)
            dfBetweenBorder = df.loc[borderFromTime:]
            #ustawiam druga granice na min lub max ktore znajduje sie pomiedzy dwoma szczytami lub dolkami
            #zmienic to border zeby nie bralo od tej co mam tylko nastpnej swieczki
            betweenBorderMax = float(dfBetweenBorder['high'].max()) #nwn czy to musi byc jako float bo jest numpy float
            borderHigh['border'] = betweenBorderMax
            print(f"borderHigh['time'] = {borderHigh['time']}")
            borderHigh['time'] = str(dfBetweenBorder[dfBetweenBorder['high'] == betweenBorderMax].index[0])
            print(f"borderHigh['time'] = {borderHigh['time']}")
            #print(dfBetweenBorder)
            print(f"borderLow['time'] = {borderLow['time']}")
            print(str(df.iloc[[-1]].index[0]))
            print(f"zmiana tempLow{trend}")
            
        borderLow['time'] = str(df.iloc[[-1]].index[0])
        
    # unidentified trend
    if tempChange == 2:
        trend = 'undefined'
    print(i)

10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
2022-11-24 17:35:00
borderHigh['time'] = 2022-11-24 18:05:00
borderHigh['time'] = 2022-11-24 18:05:00
borderLow['time'] = 2022-11-24 17:30:00
2022-11-25 01:50:00
zmiana tempLowdown
100
101
102
103
104
105
106
107
108
109
110
111
112
113
2022-11-25 01:55:00
borderHigh['time'] = 2022-11-24 18:05:00
borderHigh['time'] = 2022-11-25 02:25:00
borderLow['time'] = 2022-11-25 01:50:00
2022-11-25 03:00:00
zmiana tempLowdown
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
2022-11-25 03:05:00
borderHigh['time'] = 2022-11-25 02:25:00
borderHigh['time'] = 2022-11-25 03:50:00
borderLow['time'] = 2022-11-25 03:00:00
2022-11-25 05:35:00
zmiana tempLowdown
145
146

In [121]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 237 entries, 2022-11-24 17:30:00 to 2022-11-25 13:10:00
Freq: 5T
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    237 non-null    float64
 1   high    237 non-null    float64
 2   low     237 non-null    float64
 3   close   237 non-null    float64
 4   RSI     237 non-null    float64
dtypes: float64(5)
memory usage: 19.2 KB
